In [1]:
import pandas as pd

# Dados de treino
X_treino = pd.read_csv('Dados/dados_tratados/X_treino.csv').values
y_treino = pd.read_csv('Dados/dados_tratados/y_treino.csv').values.ravel()

# Dados de teste
X_teste = pd.read_csv('Dados/dados_tratados/X_teste.csv').values
y_teste = pd.read_csv('Dados/dados_tratados/y_teste.csv').values.ravel()

In [7]:
from sklearn.dummy import DummyClassifier
from sklearn.metrics import fbeta_score

# Instancia, ajusta e faz previsão
modelo_baseline = DummyClassifier()
modelo_baseline.fit(X_treino, y_treino)
y_previsao_baseline = modelo_baseline.predict(X_teste)

y_verdadeiro = y_teste

# Calcula f2_score
f2_score_baseline = fbeta_score(
    y_verdadeiro,          # classificações reais
    y_previsao_baseline,   # classificações previstas
    beta=2,
)

print(f'O baseline possui f2-score de {f2_score_baseline*100}%.')

O baseline possui f2-score de 0.0%.
